# **Laboratorio 5 Redes Neuronales con Pytorch**
Arancibia Aguilar Daniel Andree

Ingeniería en Ciencias de la Computación

Para las redes neuronales hare uso del siguiente dataset:

https://www.kaggle.com/datasets/anseldsouza/water-pump-rul-predictive-maintenance

Convertido a una clasificación de 9 clases

Haciendo las pruebas con diferentes hiperparametros para encontrar el mas optimo

In [109]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
# Libreria para graficos
from matplotlib import pyplot as plt

import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm  # For nice progress bar!

%matplotlib inline

In [110]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [111]:
# Here we create our simple neural network. For more details here we are subclassing and
# inheriting from nn.Module, this is the most general way to create your networks and
# allows for more flexibility. I encourage you to also check out nn.Sequential which
# would be easier to use in this scenario but I wanted to show you something that
# "always" works.
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """
        x = self.fc1(x)
        x = F.sigmoid(x)
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

**Preproceso el dataset convirtiendolo a una clasificación one vs all**

In [112]:
url = "/content/gdrive/MyDrive/SIS420/Lab5/rul_hrs.csv"
dataframe = pd.read_csv(url)
# Definir los rangos y los valores a los que se deben convertir
rango_valores = [
    (range(0, 84), 0),
    (range(84, 167), 1),
    (range(167, 251), 2),
    (range(251, 334), 3),
    (range(334, 418), 4),
    (range(418, 502), 5),
    (range(502, 585), 6),
    (range(585, 669), 7),
    (range(669, 753), 8),
    (range(753, 838), 9)
]
# Iterar sobre los rangos y valores y reemplazar los valores en la columna "rul"
for rango, valor in rango_valores:
    dataframe['rul'] = dataframe['rul'].replace(rango, valor)

print(dataframe)

        Unnamed: 0  timestamp  sensor_00  sensor_01  sensor_02  sensor_03  \
0                0      43191          2         47         53         46   
1                1      43191          2         47         53         46   
2                2      43191          2         47         53         46   
3                3      43191          2         47         53         46   
4                4      43191          2         47         53         46   
...            ...        ...        ...        ...        ...        ...   
166436      166436      43307          2         46         53         44   
166437      166437      43307          2         46         53         44   
166438      166438      43307          2         46         53         44   
166439      166439      43307          2         46         53         44   
166440      166440      43307          2         46         53         44   

        sensor_04  sensor_05  sensor_06  sensor_07  ...  sensor_42  sensor_

**Separo los valores de entrenamiento y los valores de prueba**

In [113]:
X = dataframe.drop(columns=['rul'])
y = dataframe['rul']
print(X.shape)
print(y.shape)
X=X.values
y=y.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y = np.squeeze(y)
print(X_train.shape)
print(y_train.shape)
print(X.shape)
print(y.shape)
y = np.squeeze(y)
print(y)

(166441, 52)
(166441,)
(133152, 52)
(133152,)
(166441, 52)
(166441,)
[3 3 3 ... 0 0 0]


In [114]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Crear conjuntos de datos y DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 64

# DataLoader para datos de entrenamiento
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# DataLoader para datos de prueba
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Prueba 1 de hiperparametros:**

In [115]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 52
num_classes = 10
learning_rate = 0.001
batch_size = 500000
num_epochs = 3

cpu


In [116]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

In [117]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=52, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [118]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [119]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  1%|          | 26/2081 [00:00<00:07, 259.32it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

  4%|▍         | 85/2081 [00:00<00:07, 276.93it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

  7%|▋         | 151/2081 [00:00<00:06, 306.39it/s]

...

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

 99%|█████████▉| 2056/2081 [00:06<00:00, 371.12it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


100%|██████████| 2081/2081 [00:06<00:00, 309.70it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([32, 52])


In [120]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Precisión en el entrenamiento: {p_train*100:.2f}")
print(f"Precisión en el test: {p_test*100:.2f}")

Precisión en el entrenamiento: 39.37
Precisión en el test: 39.34


# **Prueba 2 con otros hiperparametros**

In [121]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 52
num_classes = 10
learning_rate = 0.004
batch_size = 400000
num_epochs = 3

cpu


In [122]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

In [123]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=52, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [124]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [125]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  4%|▍         | 82/2081 [00:00<00:05, 394.75it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

  8%|▊         | 163/2081 [00:00<00:04, 400.92it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

 10%|▉         | 204/2081 [00:00<00:04, 381.50it/s]

...

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

 98%|█████████▊| 2037/2081 [00:05<00:00, 369.50it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])


100%|██████████| 2081/2081 [00:05<00:00, 352.06it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([32, 52])


In [126]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Precisión en el entrenamiento: {p_train*100:.2f}")
print(f"Precisión en el test: {p_test*100:.2f}")

Precisión en el entrenamiento: 27.04
Precisión en el test: 27.32


# **Prueba 3 con otros hiperparametros**

In [127]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 52
num_classes = 10
learning_rate = 0.002
batch_size = 800000
num_epochs = 3

cpu


In [128]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

In [129]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=52, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [130]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [131]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  4%|▎         | 78/2081 [00:00<00:05, 384.32it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

  8%|▊         | 160/2081 [00:00<00:04, 401.43it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

 12%|█▏        | 243/2081 [00:00<00:04, 404.11it/s]

...

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

 96%|█████████▋| 2007/2081 [00:05<00:00, 388.11it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

100%|██████████| 2081/2081 [00:05<00:00, 389.44it/s]

torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([64, 52])
torch.Size([6

In [132]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Precisión en el entrenamiento: {p_train*100:.2f}")
print(f"Precisión en el test: {p_test*100:.2f}")

Precisión en el entrenamiento: 33.74
Precisión en el test: 33.53


# **Resultado de las 3 pruebas**
## **Prueba 1 con los hiperparametros**

learning_rate = 0.001 batch_size = 500000

**Precisión en el entrenamiento: 39.37**

**Precisión en el test: 39.34**
## **Prueba 2 con los hiperparametros**

learning_rate = 0.004 batch_size = 400000

**Precisión en el entrenamiento: 27.04**

**Precisión en el test: 27.32**

## **Prueba 3 con los hiperparametros**

learning_rate = 0.002 batch_size = 800000

**Precisión en el entrenamiento: 33.74**

**Precisión en el test: 33.53**

En la primera prueba se obtuvo la mejor precisión, al aumentar el learning rate y el batch size, tambien disminuyó bastante la precisión en la segunda prueba, en la tercera prueba aumentando el learning rate y el batch size se tuvo una disminución de la precisión, con estos resultados se puede ver que un gran aumento del batch size tambien puede afectar, se debe buscar un equilibrio entre los hiperparametros